# Método Numérico de Numerov

* PET - Física UFRN
* Petiano: José Arthur de Luna Oliveira
* Data:

$\quad$ Nesse `Notebook` exploraremos o método numérico de Numerov, tal método é utilizado para resolver equações diferenciais de segunda ordem. Aqui neste `Notebook`, usaremos este método para solucionar os seguintes problemas: átomos de hidrogênio, oscilador harmônico e para um potencial bidimensional de Henon-Heiles. Vale salientar que este trabalho é baseado nos artigos *Numerical Solution of the Two-Dimensional Time Independent Schrödinger Equation with Numerov-Type Methods* **[1]** e *Applications of the Numerov method to simple quantum 
systems using Python* **[2]**.

Neste `Notebook` haverá as seguintes seções:
* Introdução;
* Problema do Átomo de Hidrogênio;
* Problema do Oscilador Harmônico;
* Problema com potencial Henon-Heiles;
* Conclusão.

## Importando Bibliotecas

In [1]:
import matplotlib.pyplot as plt
import numpy as np

## Detalhes da Biblioteca

In [2]:
%load_ext version_information
%version_information Matplotlib, Numpy

Software versions
Python 3.11.8 64bit [MSC v.1916 64 bit (AMD64)]
IPython 8.20.0
OS Windows 10 10.0.22631 SP0
Matplotlib 3.8.0
Numpy 1.26.4
Sat Apr 13 10:16:18 2024 Hora Padrão de Buenos Aires

## 1. Introdução

$\quad$ Antes de iniciarmos, vale ressaltar que essa seção é toda baseada na construção que método do artigo *Applications of the Numerov method to simple quantum systems using Python* **[2]**.

$\quad$ O principal motivo para Numerov desenvolver esse método foi que ele queria determinar as correções na trajetória do cometa Halley. Esse método, por sua vez, surgiu inicialmente para solucionar problemas de autovalores relacionados a equações diferenciais ordinárias de segunda ordem para corpos celestes. Tais equações não possuem informação da primeira derivada de uma função desconhecida, $y(x)$, em outras palavras, equações com a seguinte forma

$$ \frac{d^2y}{dx^2} = f(x, \ y) \ . \tag{1.1}$$

$\quad$ Nos métodos mais tradicionais para equações diferenciais, por exemplo, método de Euler ou o Runge-Kutta, cria-se um sistema de equações para resolvê-la como,

$$  {\begin{cases}
        z = \frac{dy}{dx}\\
        \frac{dz}{dx} = f(x, \ y)
        \end{cases}
        } \ .$$

$\quad$ No entanto, em problemas que $\frac{dy}{dx}$ é desconhecido, não pode utilizar esses métodos tradicionais para solucionar esses problemas, então deve-se usar outros métodos.

$\quad$ Para desenvolver o método que usaremos neste `Notebook`, usaremos o problema de estados ligados da Mecânica Quântica. Suponha que uma partícula de massa $m$, confinada em um potencial $V(x)$, em um dado intervalo $a<x<b$, onde as energias permitidas ($E$) e as funções de ondas correspondentes $\psi(x)$ que descrevem esses estados estacionários, satisfazem a equação de autovalor de Schrödinger

$$ \frac{d^2\psi(x)}{dx^2} + k^2(x)\psi(x) = 0 \ ,\tag{1.2}$$

na qual $k=\sqrt{2m[E - V(x)]}/\hbar$ e $\hbar = 1.055\times 10^{-34}\ J.s $ é a constante de Planck reduzida. Nesse caso, podemos estabelecer condições de continuidade para os valores $\psi$ e $d\psi/dx$ em dois ou mais pontos do domínio da função de onda. Assim, tornando desnecessária a transformação de uma equação diferencial de segunda ordem em um sistema de primeira ordem. Vale ressaltar que o método de Numerov nos permite a determinação simultânea do espectro de energia da partícula e das autofunções associadas a cada valor de energia.

### 1.2 Encontrando a Fórmula de Numerov

$\quad$ O método numérico de Numerov é um método interativo, assim a solução deve ser conhecida em dois pontos subsequentes no intervalo $[a, \ b]$, ou seja, $\psi(x - \delta)$ e $\psi(x)$ devem ser conhecidos, de forma que $\delta$ é um valor muito pequeno, e será o passo de integração do nosso algoritmo.

$\quad$ Para construir o nosso algoritmo, começaremos expandindo em série de Taylor o $\psi(x \pm \delta)$ até a quarta ordem,

$$ \psi(x \pm \delta) = \psi(x) \pm \delta \psi^{(1)}(x) + \frac{\delta^2}{2}\psi^{(2)}(x) \pm \frac{\delta^3}{6}\psi^{(3)}(x) + \frac{\delta^4}{24}\psi^{(4)}(x) \ , \tag{1.3}$$

somando os termos $\psi(x + \delta)$ e $\psi(x-\delta)$, temos

$$ \psi(x+\delta) + \psi(x-\delta) = \psi(x) + \delta \psi^{(1)}(x) + \frac{\delta^2}{2}\psi^{(2)}(x) + \frac{\delta^3}{6}\psi^{(3)}(x) + \frac{\delta^4}{24}\psi^{(4)}(x) + \psi(x) - \delta \psi^{(1)}(x) + \frac{\delta^2}{2}\psi^{(2)}(x) - \frac{\delta^3}{6}\psi^{(3)}(x) + \frac{\delta^4}{24}\psi^{(4)}(x)$$

$$ \implies \psi(x+\delta) + \psi(x-\delta) = 2\psi(x) + \delta^2\psi^{(2)}(x)+ \frac{\delta^4}{12}\psi^{(4)}(x)  \implies \psi(x+\delta) + \psi(x-\delta) - 2\psi(x) = \delta^2\psi^{(2)}(x)+ \frac{\delta^4}{12}\psi^{(4)}(x)$$

$$ \therefore \boxed{ \frac{\psi(x+\delta) + \psi(x-\delta) - 2\psi(x)}{\delta^2} = \left(1 + \frac{\delta^2}{12}\frac{d^2}{dx^2} \right)\psi^{(2)}(x)} \ .\tag{1.4}$$

$\quad$ Escrevendo a Equação **(1.2)** da seguinte forma, $\frac{d^2\psi(x)}{dx^2} = - k^2(x)\psi(x)$, podemos substituir isso na Equação **(1.4)**, assim

$$ \frac{\psi(x+\delta) + \psi(x-\delta) - 2\psi(x)}{\delta^2} = \left(1 + \frac{\delta^2}{12}\frac{d^2}{dx^2} \right)\left[-k^2(x)\psi(x)\right]$$

$$ \implies \frac{\psi(x+\delta) + \psi(x-\delta) - 2\psi(x)}{\delta^2} = -k^2(x)\psi(x) - \frac{\delta^2}{12}\frac{d^2}{dx^2}\left[k^2(x)\psi(x)\right]\ , \tag{1.5}$$

agora faremos a seguinte manipulação na Equação **(1.4)**,

$$ \frac{\psi(x+\delta) + \psi(x-\delta) - 2\psi(x)}{\delta^2} = \psi^{(2)}(x)+ \frac{\delta^2}{12}\psi^{(4)}(x)$$

$$\implies \psi^{(2)}(x) = \frac{\psi(x+\delta) + \psi(x-\delta) - 2\psi(x)}{\delta^2} - \frac{\delta^2}{12}\psi^{(4)}(x) , $$

vamos fazer a seguinte troca $\psi(x) \rightarrow k^2(x)\psi(x)$, então

$$ \frac{d^2}{dx^2}[k^2(x)\psi(x)] = \frac{k^2(x+\delta)\psi(x+\delta) + k^2(x-\delta)\psi(x-\delta) - 2k^2(x)\psi(x)}{\delta^2} - \frac{\delta^2}{12}[k^2(x)\psi(x)]^{(4)} \ ,$$

substituindo esse resultado na Equação **(1.5)**, temos

$$\frac{\psi(x+\delta) + \psi(x-\delta) - 2\psi(x)}{\delta^2} =-k^2(x)\psi(x) - \frac{\delta^2}{12}\left[\frac{k^2(x+\delta)\psi(x+\delta) + k^2(x-\delta)\psi(x-\delta) - 2k^2(x)\psi(x)}{\delta^2}\right] - \frac{\delta^4}{144}[k^2(x)\psi(x)]^{(4)} \ ,$$

considerando que $\mathcal{O(\delta^4)} = - \frac{\delta^4}{144}[k^2(x)\psi(x)]^{(4)}$, então

$$\frac{\psi(x+\delta) + \psi(x-\delta) - 2\psi(x)}{\delta^2} =-k^2(x)\psi(x) - \frac{\delta^2}{12}\left[\frac{k^2(x+\delta)\psi(x+\delta) + k^2(x-\delta)\psi(x-\delta) - 2k^2(x)\psi(x)}{\delta^2}\right] + \mathcal{O(\delta^4)}$$

$$\implies \psi(x+\delta) + \psi(x-\delta) - 2\psi(x) =-\delta^2 k^2(x)\psi(x) - \frac{\delta^2}{12}\left[k^2(x+\delta)\psi(x+\delta) + k^2(x-\delta)\psi(x-\delta) - 2k^2(x)\psi(x)\right] + \mathcal{O(\delta^4)}$$

$$\implies \psi(x+\delta) + \frac{\delta^2}{12}k^2(x+\delta)\psi(x+\delta) = 2\psi(x) -\delta^2 k^2(x)\psi(x) + \frac{\delta^2}{12}2k^2(x)\psi(x) -\psi(x-\delta) - \frac{\delta^2}{12} k^2(x-\delta)\psi(x-\delta) + \mathcal{O(\delta^4)}$$

$$\implies \left[1 + \frac{\delta^2}{12}k^2(x+\delta)\right]\psi(x+\delta) = \left[2  + \frac{\delta^2k^2(x) -6\delta^2 k^2(x)}{6}\right]\psi(x) -\left[1 + \frac{\delta^2}{12} k^2(x-\delta)\right]\psi(x-\delta) + \mathcal{O(\delta^4)}$$

$$\implies \left[1 + \frac{\delta^2}{12}k^2(x+\delta)\right]\psi(x+\delta) = 2\left[1  - \frac{5\delta^2}{12} k^2(x)\right]\psi(x) -\left[1 + \frac{\delta^2}{12} k^2(x-\delta)\right]\psi(x-\delta) + \mathcal{O(\delta^4)} \ ,$$

consideraremos $\mathcal{O(\delta^4)} \approx 0$, assim

$$\left[1 + \frac{\delta^2}{12}k^2(x+\delta)\right]\psi(x+\delta) = 2\left[1  - \frac{5\delta^2}{12} k^2(x)\right]\psi(x) -\left[1 + \frac{\delta^2}{12} k^2(x-\delta)\right]\psi(x-\delta)$$

$$\therefore \ \boxed{\psi(x+\delta) = \frac{2\left[1  - \frac{5\delta^2}{12} k^2(x)\right]\psi(x) -\left[1 + \frac{\delta^2}{12} k^2(x-\delta)\right]\psi(x-\delta)}{\left[1 + \frac{\delta^2}{12}k^2(x+\delta)\right]}} \ . \tag{1.6}$$

$\quad$ Nosso objetivo é resolver uma equação de autovalor, a técnica de integração numérica da equação de Schrödinger unidimensional para uma partícula em um poço, precisará de um chute inicial para os autovalores (a energia $E$) e os dois pontos respectivos das autofunções. Para a escolha do valor arbitrário da energia $E$, devemos lembrar da relação de incerteza de Heisenberg, que diz que a energia $E$ de uma partícula em um poço de potencial $V(x)$ deve ser maior que o valor mínimo do poço, ou seja, $E_{initial} = V_{min} + \Delta E$, em que $\Delta E > 0$.

$\quad$ A escolha desse valor determinará os pontos do retorno da partícula, que para encontrá-los basta igualar o valor arbitrário $E$ e o potencial $V(x)$, fazendo isso encontrará dois pontos de retorno da partícula $x_r$ e $x_l$. Na Mecânica Clássica, a região $[x_r, \ x_l]$ será a única região que a partícula poderá ficar, em outras palavras, a partícula é restrita a se manter nessa região, enquanto as regiões além desses pontos ($x<x_r$ e $x>x_l$ ) são chamadas na mecânica clássica de regiões proibidas.

$\quad$ No entanto na Mecânica Quântica é possível da partícula se encontrar além dos pontos de retorno, ou seja, a equação Schrödinger possui solução nessas regiões. Assim, cada valor de energia possui uma autofunção associada, as quais terão solução na região proibida, no entanto, ela tenderá a zero.
Dessa forma, esses pontos, onde a autofunção se anula, são os pontos de contorno $a$ e $b$, onde $b>a$, do domínio de integração.

$\quad$ O método de Numerov requer um esquema de interação usando a Equação **(1.6)**, que a partir do ponto $a$ para a direita, criará uma função que até um ponto ($x_{match}$), e do ponto $b$ para a esquerda, criará outra função até esse mesmo ponto. Sendo assim, teremos duas funções $\psi^{\ l}(x)$ e $\psi^{\ r}(x)$, a primeira virá da esquerda e a outra da direita, a partir dos pontos $a$ e $b$, respectivamente.

### 1.3 Construindo o Algoritmo

$\quad$ Diante do que foi dito, devemos escolher um valor inicial para $E$, obter duas soluções, uma pela esquerda, $\psi^{\ l}(x)$ e uma pela direita, $\psi^{\ l}(x)$, nas quais devemos começar com os dois pontos consecutivos iniciais e aplicar na Equação **(1.6)** para obter o resto da solução. Em seguida, explicitaremos o que deve ser feito.

#### Solução pela esquerda ($x < x_{match}$)

$\quad$ Dado o chute inicial para a energia, deve-se encontrar os pontos de retorno para aquela energia, assim podemos escolher um desses pontos como o $x_{match}$. Em seguida, devemos definir o dois pontos iniciais consecutivos da solução $\psi^{\ l}(x)$ como

$$  {\begin{cases}
        \psi^{\ l}(a) = 0\\
        \psi^{\ l}(a+\delta) = \delta^{\ l}, \ (\delta^{\ l} \ll 1)
        \end{cases}
        } \ ,$$

as quais, o $\delta$ é o passo de integração e $\delta^l$ é um valor muito pequeno. Note que podemos igualar o $\psi^l(a+\delta)$ a um valor muito pequeno, pois como estamos dividindo a função em intervalos de muito pequenos ($\delta$), o próximo ponto consecutivo à zero ($\psi^l=0$) será um número muito próximo de zero. 

$\quad$ Definido esse pontos iniciais, basta aplicar a Equação **(1.6)**, que obtêm-se cada ponto da solução sequencialmente até $x_{match}$.

### Solução pela Direita ($x>x_{match}$)

$\quad$ Análogo ao processo explicado anteriormente, com o mesmo valor de energia e o mesmo $x_{match}$, definimos os dois pontos iniciais consecutivos da solução $\psi^{\ r}(x)$ como

$$  {\begin{cases}
        \psi^{\ r}(b) = 0\\
        \psi^{\ r}(b-\delta) = \delta^{\ r}, \ (\delta^{\ r} \ll 1)
        \end{cases}
        } \ .$$

$\quad$ Dessa forma, aplicando a Equação **(1.6)** obtêm a solução $\psi^{\ r}(x)$ até $x_{match}$.

$\quad$ Note que a construção das duas soluções terminam em $x_{match}$ e nesse ponto estas devem ser iguais nesse ponto, garantindo a continuidade da solução,

$$\psi^{\ l}(x_{match}) = \psi^{\ r}(x_{match}) \ ,\tag{1.7}$$

no entanto nem sempre as soluções possíveis atendem a esse ponto, então para garantir a continuidade da solução, redefiniremos o $\psi^{\ l}$ da seguinte forma,

$$ \psi^{\ l}(x) \rightarrow \psi^{\ l}(x)\frac{\psi^{\ r}(x_{match})}{\psi^{\ l}(x_{match})} \ ,\tag{1.8}$$

no qual $a \leq x \leq x_{match}$.

# Referências

**[1]** Kalogiratou, Z., et al. “Numerical Solution of the Two-Dimensional Time Independent Schrödinger Equation with Numerov-Type Methods.” Journal of Mathematical Chemistry, vol. 37, no. 3, Apr. 2005, pp. 271–279, https://doi.org/10.1007/s10910-004-1469-1. Accessed 29 Mar. 2024.

**[2]** Caruso, Francisco, et al. “Applications of the Numerov Method to Simple Quantum Systems Using Python.” Revista Brasileira de Ensino de Física, vol. 44, no. e20220098, 2022, https://doi.org/10.1590/1806-9126-rbef-2022-0098. Accessed 29 Mar. 2024.

**[3]** Hjorth-Jensen, Morten. Computational Physics. University of Oslo, Aug. 2014.

**[4]** Investigação da energia de ligação entre átomos utilizando o potencial de Lennard-Jones: https://github.com/PETfisicaUFRN/PET.py/blob/main/Notebooks/Investiga%C3%A7%C3%A3o%20da%20energia%20de%20liga%C3%A7%C3%A3o%20entre%20%C3%A1tomos%20utilizando%20o%20potencial%20de%20Lennard-Jones.ipynb

**[5]** Newman, Mark. Computacional Physics. 2012. University of Michigan, 2013.

**[6]** Giordano, N. J., & Nakanishi, H. (2006). Computational physics.